<a href="https://colab.research.google.com/github/juned202/M.Tech.-Thesis-JUNED-/blob/main/Scie_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credit: https://www.kaggle.com/code/isaacmg/scibert-embeddings/notebook

For instance, the embedding method displayed here is a simple MEAN over all the words in the text passage.

In [3]:
!pip install transformers
!wget -O scibert_uncased.tar https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar
!tar -xvf scibert_uncased.tar
import torch
from transformers import BertTokenizer, BertModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-07-31 08:54:37--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.92.148.168
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.92.148.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 442460160 (422M) [application/x-tar]
Saving to: ‘scibert_uncased.tar’

scibert_uncased.tar 100%[===================>] 421.96M  21.7MB/s    in 21s     

2022-07-31 08:54:59 (20.1 MB/s) - ‘scibert_uncased.tar’ saved [442460160/442460160]

scibert_scivocab_uncased/
scibert_scivocab_uncased/vocab.txt
scibert_scivocab_uncased/pytorch_model.bin
scibert_scivocab_uncased/config.json


We will use the SciBERT Vocab uncased model as that is what is recommended on the official GitHub Page

In [4]:
model_version = 'scibert_scivocab_uncased'
do_lower_case = True
model = BertModel.from_pretrained(model_version)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)

Some weights of the model checkpoint at scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokeni

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
def embed_text(text, model):
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)  # Batch size 1
    outputs = model(input_ids)
    last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
    return last_hidden_states 

def get_similarity(em, em2):
    return cosine_similarity(em.detach().numpy(), em2.detach().numpy())

In [6]:
# We will use a mean of all word embeddings. To do that we will take mean over dimension 1 which is the sequence length.
Research_em = embed_text("Research", model).mean(1)
computer_em = embed_text("computer science articles", model).mean(1)
Rec_em = embed_text("Recommender System", model).mean(1)
medi_em = embed_text("biotech", model).mean(1)
ir_em= embed_text("scholarly", model).mean(1)
print("Similarity for Research and computer:" + str(get_similarity(Research_em, computer_em)))
print("Similarity for Research and Rec:" + str(get_similarity(Research_em, Rec_em)))
print("Similarity for Research and medi:" + str(get_similarity(Research_em,medi_em)))
print("Similarity for Research and ir:" + str(get_similarity(Research_em, ir_em)))

Similarity for Research and computer:[[0.659531]]
Similarity for Research and Rec:[[0.67065305]]
Similarity for Research and medi:[[0.667645]]
Similarity for Research and ir:[[0.82327175]]


Let's now look at visualizing some of these vectors with U-Map. I'm choosing U-Map here due to the high-dimensionality of the data (768-D

In [7]:
!pip install umap-learn
import umap
reducer = umap.UMAP()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import os
import json 
def make_the_embeds(number_files, start_range=0, 
                    the_path='/content/drive/Shareddrives/M.Tech.Thesis(Juned)/DATASET/es_demo.json'):
  
    data = []
    for line in open(the_path, 'r'):
        data.append(json.loads(line))
        num=len(data)

    # data preprocessing

    doc_preprocess = []
    for x in range(num):
        text=(data[x]['_source']['title'])
        doc_preprocess.append((text))





    title_embedding_list = [] 
    title_list = []
    for i in range(start_range, number_files):
        # file_name = the_list[i]
        # final_path = os.path.join(the_path, file_name)
        
        try:
            tensor, title = make_data_embedding(doc_preprocess[i])
            title_embedding_list.append(tensor)
            title_list.append(title)
        except:
            print("Invalid title/abstract")
    return torch.cat(title_embedding_list, dim=0), title_list
        
def make_data_embedding(article_data, method="mean", dim=1):
    data = article_data
    # for key in data_keys:
    #     data = data[key]
    text = embed_text(data, model)
    if method == "mean":
        return text.mean(dim), data
    
embed_list, title_list = make_the_embeds(200)
red = reducer.fit_transform(embed_list.detach().numpy())

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


I found 200 to be a good chunk size for running quick analysis; doing a full plot can get kind of crowded and is slow to compute.

In [9]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.palettes import Spectral10, Category20c
from bokeh.palettes import magma
import pandas as pd
output_notebook()

In [10]:
def make_plot(red, title_list, number=200, color = True, color_mapping_cat=None, color_cats = None, bg_color="white"):   
    digits_df = pd.DataFrame(red, columns=('x', 'y'))
    if color_mapping_cat:
        digits_df['colors'] = color_mapping_cat
    digits_df['digit'] = title_list
    datasource = ColumnDataSource(digits_df)
    plot_figure = figure(
    title='UMAP projection of the article title embeddings',
    plot_width=890,
    plot_height=600,
    tools=('pan, wheel_zoom, reset'),
    background_fill_color = bg_color
    )
    plot_figure.legend.location = "top_left",
    plot_figure.add_tools(HoverTool(tooltips="""
    <div>
    <div>
        <img src='@image' style='float: left; margin: 5px 5px 5px 5px'/>
    </div>
    <div>
        <span style='font-size: 10px; color: #224499'></span>
        <span style='font-size: 10px'>@digit</span>
    </div>
    </div>
    """))
    if color:   
        color_mapping = CategoricalColorMapper(factors=title_list, palette=magma(number))
        plot_figure.circle(
            'x',
            'y',
            source=datasource,
            color=dict(field='digit', transform=color_mapping),
            line_alpha=0.6,
            fill_alpha=0.6,
            size=7
        )
        show(plot_figure)
    elif color_mapping_cat:
        color_mapping = CategoricalColorMapper(factors=color_cats, palette=magma(len(color_cats)+2)[2:])
        plot_figure.circle(
            'x',
            'y',
            source=datasource,
            color=dict(field='colors', transform=color_mapping),
            line_alpha=0.6,
            fill_alpha=0.6,
            size=8,
            legend_field='colors'
        )
        show(plot_figure)
    else:
        
        plot_figure.circle(
            'x',
            'y',
            source=datasource,
            color=dict(field='digit'),
            line_alpha=0.6,
            fill_alpha=0.6,
            size=7
        )
        show(plot_figure)
    
make_plot(red, title_list, number=200)

/usr/local/lib/python3.7/dist-packages/bokeh/models/plots.py:766: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


There do seem to be a few interesing patterns when analyizng with U-Map. However, I believe more sophisticated methods could definitely improve the clustering of groups. Let's examine another chunk

In [11]:
embed_list2, title_list2 = make_the_embeds(401, 201)
red2 = reducer.fit_transform(embed_list.detach().numpy())
print(len(title_list2))
make_plot(red2, title_list2, number=198)

200


/usr/local/lib/python3.7/dist-packages/bokeh/models/plots.py:766: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


**Search Attempts on Titles**

We are now going to build a very simple semantic search engine on the titles of the articles. This should in theory return the most similar articles for a given query. However, due to memory constraints I'm only going to run it on 200 titles instead of the whole corpus.

In [13]:
import collections
q1 = "Research on NOFRF Entropy-Based Detection Method for Early Damage of Pillar Porcelain Insulator"
search_terms = embed_text(q1, model).mean(1)

In [22]:
search_terms.shape

torch.Size([1, 768])

In [14]:
def top_n_closest(search_term_embedding, title_embeddings, original_titles, n=10):
    proximity_dict = {}
    i = 0 
    for title_embedding in title_embeddings:
        proximity_dict[original_titles[i]] = {"score": get_similarity(title_embedding.unsqueeze(0),search_term_embedding), 
                                              "title_embedding":title_embedding.unsqueeze(0)}
        i+=1
    order_dict = collections.OrderedDict({k: v for k, v in sorted(proximity_dict.items(), key=lambda item: item[1]["score"])})
    proper_list = list(order_dict.keys())[-n:]
    return proper_list, order_dict

In [15]:
top_titles, order_dict = top_n_closest(search_terms, embed_list2, title_list+title_list2)

In [16]:
top_titles

['Quantized Kalman Filter-Based Pattern Matching for Detection and Tracking of Moving Objects',
 'Amniotic Fluid Segmentation by Pixel Classification in B-Mode Ultrasound Image for Computer Assisted Diagnosis',
 'A Novel Algorithm for Automatic Text Summarization System Using Lexical Chain',
 'Research on Traditional Structure Optimization in Cloud Computing Environment',
 'An End-User Pipeline for Scraping and Visualizing Semi-Structured Data over the Web',
 'Optical Diagnosis of Colorectal Polyps: Recent Developments',
 'Visual SLAM in Human Populated Environments: Exploring the Trade-off between Accuracy and Speed of YOLO and Mask R-CNN',
 'Design and Implementation of Color Detection Technique using Fuzzy Logic',
 'Adaptable mobile cloud computing environment with code transfer based on machine learning',
 'Retweet Prediction Using Context-Aware Coupled Matrix-Tensor Factorization']

In [18]:
q2 = "Quantized Kalman Filter-Based Pattern Matching for Detection and Tracking of Moving Objects"

In [19]:
search_terms2 = embed_text(q2, model).mean(1)
top_titles2, order_dict1 = top_n_closest(search_terms2, embed_list2, title_list + title_list2)
top_titles2

["A Dynamic Microtask Approach to Collecting and Organizing Citizens' Opinions",
 'Bridging the Gap between High-Performance, Cloud and Service-Oriented Computing',
 'A Novel fuzzy frame selection based watermarking scheme for MPEG-4 videos using Bi-directional extreme learning machine',
 'Visual SLAM in Human Populated Environments: Exploring the Trade-off between Accuracy and Speed of YOLO and Mask R-CNN',
 'A new metaheuristic optimization method: the algorithm of the innovative gunner (AIG)',
 'Flower snark and related graph’s reverse super edge – Magic labelings',
 'Convolutional Neural Network for Satellite Image Classification',
 'Two Experiments Comparing Two Four-Step EPMcreate-Based Creativity Techniques for Requirements Elicitation',
 'Correlation effects and configuration of a one-dimensional bipolaron',
 'Video Abnormal Event Detection Based on ELM']